In [ ]:
import pandas as pd

pd.options.display.max_columns = 999

In [ ]:
emp = pd.read_parquet("dados/empresas.parquet")
estb = pd.read_parquet("dados/estabelecimentos_reduzido.parquet")
final = estb.merge(emp, on="cnpj_raiz")
cnae = pd.read_parquet("dados/cnaes.parquet")
final = final.merge(cnae, left_on="cnae_fiscal", right_on="cnae")
final.to_parquet("dados/estabelecimentos_completo_com_cnae.parquet")

In [4]:
final = pd.read_parquet("estabelecimentos_completo_com_cnae.parquet")

In [ ]:
paises = pd.read_csv("F.K03200$Z.D50208.PAISCSV", encoding="iso-8859-1", sep=";", header=None)
natju = pd.read_csv("F.K03200$Z.D50208.NATJUCSV", encoding="iso-8859-1", sep=";", header=None)
motiv = pd.read_csv("F.K03200$Z.D50208.MOTICSV", encoding="iso-8859-1", sep=";", header=None)

In [ ]:
paises.columns = ["pais", "nome_pais"]
natju.columns = ["natju", "nome_natureza_juridica"]
motiv.columns = ["motivo", "nome_motivo"]

In [ ]:
final = final.merge(paises, left_on="pais", right_on="pais", how="left")
final = final.merge(natju, left_on="naturezajuridica", right_on="natju", how="left")
final = final.merge(motiv, left_on="motivo_situacao_cadastral", right_on="motivo", how="left")

In [ ]:
layout = [
    "cnpj_raiz",
    "cnpj",
    "razaosocial",
    "nome_fantasia",
    "identificador_matriz_filial",
    "situacao_cadastral",
    "data_situacao_cadastral",
    "motivo_situacao_cadastral",
    "nome_cidade_exterior",
    "data_inicio_atividade",
    "uf",
    "naturezajuridica",
    "qualificacao",
    "capital_social",
    "porte",
    "ente_federativo",
    "cnae_fiscal",
    "descricao_cnae",
    "nome_pais",
    "pais",
    "nome_natureza_juridica",
    "nome_motivo",
]

In [ ]:
final = final[layout]
final['descricao_cnae'] = final['descricao']

In [48]:
final['capital_social'] = final['capital_social'].apply(lambda x: x.replace(',','.')).astype(float)

In [49]:
final.head()

,cnpj_raiz,cnpj,razaosocial,nome_fantasia,identificador_matriz_filial,situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_cidade_exterior,data_inicio_atividade,...,qualificacao,capital_social,porte,ente_federativo,cnae_fiscal,descricao_cnae,nome_pais,pais,nome_natureza_juridica,nome_motivo
0,52636412,52636412012637,PB ADMINISTRADORA DE ESTACIONAMENTOS LTDA,ROYAL PALM CONTEMPORANEO,2,2,20190711,0,None,20190711,...,5,399039118.0,5.0,None,5223100,Estacionamento de veículos,NaN,NaN,Sociedade Empresária Limitada,SEM MOTIVO
1,52636412,52636412014095,PB ADMINISTRADORA DE ESTACIONAMENTOS LTDA,ESTACIONAMENTO PRUDENSHOPPING,2,2,20200129,0,None,20200129,...,5,399039118.0,5.0,None,5223100,Estacionamento de veículos,NaN,NaN,Sociedade Empresária Limitada,SEM MOTIVO
2,52636412,52636412014257,PB ADMINISTRADORA DE ESTACIONAMENTOS LTDA,ESTACIONAMENTO CENTRO MEDICO OPTY,2,2,20190923,0,None,20190923,...,5,399039118.0,5.0,None,5223100,Estacionamento de veículos,NaN,NaN,Sociedade Empresária Limitada,SEM MOTIVO
3,52636412,52636412014338,PB ADMINISTRADORA DE ESTACIONAMENTOS LTDA,ESTACIONAMENTO DO HOTEL PLAZA SAO RAFAEL,2,2,20200316,0,None,20200316,...,5,399039118.0,5.0,None,5223100,Estacionamento de veículos,NaN,NaN,Sociedade Empresária Limitada,SEM MOTIVO
4,52636412,52636412014761,PB ADMINISTRADORA DE ESTACIONAMENTOS LTDA,TERRENO MACKENZIE,2,2,20201104,0,None,20201104,...,5,399039118.0,5.0,None,5223100,Estacionamento de veículos,NaN,NaN,Sociedade Empresária Limitada,SEM MOTIVO


In [ ]:
porte = {5: "demais", 3: "pequeno", 1: "micro", 0: "não informado"}
final["porte"] = final["porte"].map(porte)

flag_filial = {1: "matriz", 2: "filial"}
final["identificador_matriz_filial"] = final["identificador_matriz_filial"].map(
    flag_filial
)

situacao_cadastral = {2: "ativa", 8: "baixada", 3: "suspensa", 4: "inapta", 1: "nula"}

final["situacao_cadastral"] = final["situacao_cadastral"].map(situacao_cadastral)

In [60]:
final.to_parquet("estabelecimentos_full.parquet")

In [ ]:
final.drop(columns=["naturezajuridica", "motivo_situacao_cadastral"], inplace=True)
final.drop(columns=["pais"], inplace=True)
final = final.query("situacao_cadastral == 'ativa'").drop(
    columns=["situacao_cadastral"]
)
final.drop(columns=["cnpj_raiz"], inplace=True)

In [ ]:
qualificacoes = pd.read_csv(
    "F.K03200$Z.D50208.QUALSCSV", encoding="iso-8859-1", sep=";", header=None
)
qualificacoes.columns = ["qualificacao", "nome_qualificacao"]
final = final.merge(qualificacoes, on="qualificacao", how="left")
final.drop(columns=["qualificacao"], inplace=True)
final = final.query("porte != 'micro'")

In [22]:
final.to_parquet("estabelecimentos_full_ativos_pequeno_port_ou_superior.parquet")